Створіть програму на Python, яка імітує менеджер завдань і використовує мультитредінг. Програма має дозволяти користувачеві додавати завдання (прості математичні обчислення, робота з текстом тощо) і виконувати їх паралельно у різних потоках. Забезпечте коректну синхронізацію між потоками, щоб уникнути проблем зі змаганням за ресурси

In [ ]:
import tkinter as tk
import threading
import time
import random

class TaskExecutor:
    def __init__(self, app):
        self.tasks = []
        self.app = app
        self.lock = threading.Lock()

    def add_task(self, task):
        with self.lock:
            self.tasks.append({"text": task, "completed": tk.BooleanVar(value=False), "color": self.generate_random_color()})
            self.app.root.after(0, self.app.update_task_listbox)

    def generate_random_color(self):
        return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    def execute_task(self, task):
        time.sleep(2)
        with self.lock:
            task["completed"].set(True)
            self.app.root.after(0, self.app.update_task_listbox)

    def execute_tasks(self):
        threads = []
        with self.lock:
            for task in self.tasks:
                if not task["completed"].get():
                    thread = threading.Thread(target=self.execute_task, args=(task,))
                    threads.append(thread)
                    thread.start()

        for thread in threads:
            thread.join()

class TaskManagerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Task Manager")

        self.task_executor = TaskExecutor(self)

        self.task_canvas = tk.Canvas(root)
        self.task_canvas.pack(pady=10)

        self.task_entry = tk.Entry(root, width=30)
        self.task_entry.pack(pady=10)

        self.add_task_button = tk.Button(root, text="Add Task", command=self.add_task)
        self.add_task_button.pack(pady=5)

    def add_task(self):
        task_text = self.task_entry.get().strip()
        if task_text:
            threading.Thread(target=self.add_task_and_clear_entry, args=(task_text,)).start()

    def add_task_and_clear_entry(self, task_text):
        self.task_executor.add_task(task_text)
        
        self.root.after(0, self.clear_entry)

    def clear_entry(self):
        self.task_entry.delete(0, tk.END)

    def update_task_listbox(self):
        self.task_canvas.delete("all")
        with self.task_executor.lock:
            for i, task in enumerate(self.task_executor.tasks, start=1):
                checkbutton = tk.Checkbutton(self.task_canvas, variable=task["completed"],
                                             command=lambda t=task: self.update_completed_label(t),
                                             state=tk.DISABLED if task["completed"].get() else tk.NORMAL)
                self.task_canvas.create_window(10, i * 25, anchor=tk.W, window=checkbutton)

                label_text = f"{i}. {task['text']}"
                if task["completed"].get():
                    label_text += " - completed"

                label = tk.Label(self.task_canvas, text=label_text, bg=task["color"])
                self.task_canvas.create_window(30, i * 25, anchor=tk.W, window=label)

    def execute_task(self, task):
        threading.Thread(target=self.task_executor.execute_task, args=(task,)).start()

    def update_completed_label(self, task):
        if not task["completed"].get():
            label_text = f"{task['text']}"
            label = tk.Label(self.task_canvas, text=label_text, bg=task["color"])
            self.task_canvas.create_window(30, (self.task_executor.tasks.index(task) + 1) * 25, anchor=tk.W, window=label)
        else:
            self.update_task_listbox()

if __name__ == "__main__":
    root = tk.Tk()
    app = TaskManagerApp(root)
    root.mainloop()